In [1]:
import sys
import os
import boto3

sys.path.append(
    os.path.abspath(
        os.path.join(os.getcwd(), '../credentials')
    )
)


In [2]:
from credentials import ORACLE_S3_ACCESS_KEY, ORACLE_S3_SECRET_KEY, ORACLE_S3_ENDPOINT, ORACLE_REGION, ORACLE_INGEST_BUCKET

In [3]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=ORACLE_S3_ACCESS_KEY,
    aws_secret_access_key=ORACLE_S3_SECRET_KEY,
    endpoint_url=ORACLE_S3_ENDPOINT
)

In [4]:
objects = s3_client.list_objects_v2(Bucket=ORACLE_INGEST_BUCKET)

In [5]:
objects.get("Contents.Etag",[])

[]

In [6]:
from credentials import MILVUS_HOST, MILVUS_PORT, COLLECTION_NAME

In [7]:
from pymilvus import MilvusClient, Collection

# Authentication not enabled
client = MilvusClient("http://"+MILVUS_HOST+":"+MILVUS_PORT, db_name="default")

In [8]:
client.list_collections()

['docs', 'energy_vector']

In [9]:


if not client.has_collection(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        dimension=768,
        metric_type="COSINE",
        auto_id=True,
        enable_dynamic_field=True
    )
    print(f"Collection '{COLLECTION_NAME}' created.")
else:
    print(f"Collection '{COLLECTION_NAME}' already exists.")

client.load_collection(COLLECTION_NAME)

Collection 'energy_vector' already exists.


In [11]:
from typing import Iterable, Dict, Any, List, Generator
from pymilvus import MilvusClient

def _norm_etag(x: str) -> str:
    return str(x).strip('"').strip("'") if x is not None else x

def _batched(seq: List[str], n: int) -> Iterable[List[str]]:
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

def _query_existing_etags(client: MilvusClient, collection: str, etags: List[str], etag_field="etag", batch_size=500) -> set:
    """Check existence in Milvus in IN-batches; returns a set of found etags."""
    found = set()
    for batch in _batched(etags, batch_size):
        in_list = ",".join([f'"{e}"' for e in batch])
        filt = f'{etag_field} in [{in_list}]'
        rows = client.query(
            collection_name=collection,
            filter=filt,
            output_fields=[etag_field],
            limit=len(batch)
        )
        for r in rows:
            val = r.get(etag_field)
            if val is not None:
                found.add(_norm_etag(val))
    return found

def iter_new_objects_by_etag(
    s3_client,
    bucket: str,
    prefix: str,
    milvus_client: MilvusClient,
    collection_name: str,
    etag_field: str = "etag",
    page_batch_check: int = 750,
) -> Generator[Dict[str, Any], None, None]:
    """
    Stream pages from S3/OCI and yield only objects whose ETag is NOT already in Milvus.
    Memory-friendly: handles a single page at a time.
    """
    paginator = s3_client.get_paginator("list_objects_v2")
    pages = paginator.paginate(Bucket=bucket, Prefix=prefix or "")


    milvus_client.load_collection(collection_name)

    for page in pages:
        contents = page.get("Contents", []) or []
        if not contents:
            continue

        etag_to_objs: Dict[str, List[Dict[str, Any]]] = {}
        for obj in contents:
            etg = _norm_etag(obj.get("ETag"))
            if not etg:
                continue
            etag_to_objs.setdefault(etg, []).append(obj)

        unique_etags = list(etag_to_objs.keys())

        existing = set()
        for sub in _batched(unique_etags, page_batch_check):
            existing |= _query_existing_etags(
                milvus_client, collection_name, sub, etag_field=etag_field, batch_size=page_batch_check
            )

        for etg, objs in etag_to_objs.items():
            if etg not in existing:
                for obj in objs:
                    yield obj


In [ ]:
import io
import os
from typing import Tuple, List, Dict, Any
from pypdf import PdfReader


def _is_pdf_key(key: str) -> bool:
    return key.lower().endswith(".pdf")


def read_pdf_from_s3_bytes(s3_client, bucket: str, key: str, max_pages: int = None) -> Tuple[str, List[str], Dict[str, Any]]:
    """
    Download the object into memory (bytes) and extract text.
    Returns: (full_text, pages_list, meta)
    """
    resp = s3_client.get_object(Bucket=bucket, Key=key)
    body = resp["Body"].read()  # bytes
    reader = PdfReader(io.BytesIO(body))

    try:
        if getattr(reader, "is_encrypted", False):
            try:
                reader.decrypt("")
            except Exception:
                pass
    except Exception:
        pass

    pages, limit = [], (max_pages or 10**9)
    for i, page in enumerate(reader.pages):
        if i >= limit:
            break
        txt = page.extract_text() or ""
        pages.append(txt)

    full_text = "\n\n".join(pages)
    meta = {
        "n_pages_total": len(reader.pages),
        "n_pages_read": len(pages),
        "content_length": resp.get("ContentLength"),
        "content_type": resp.get("ContentType"),
        "key": key,
        "bucket": bucket,
    }
    return full_text, pages, meta


In [ ]:
new_objs_iter = iter_new_objects_by_etag(
    s3_client=s3_client,
    bucket=ORACLE_INGEST_BUCKET,
    prefix="",
    milvus_client=client,
    collection_name=COLLECTION_NAME,
    etag_field="Etag",
    page_batch_check=5
)

for obj in new_objs_iter:
    key  = obj["Key"]
    etag = obj["ETag"].strip('"')
    print(key," : ",etag)


    if not _is_pdf_key(key):
        continue

    try:
        text, pages, meta = read_pdf_from_s3_bytes(s3_client, ORACLE_INGEST_BUCKET, key)
        print(f"[PDF] {key} | etag={etag} | pages_read={meta['n_pages_read']}/{meta['n_pages_total']}")

        print(text)

    except Exception as e:
        print(f"[skip] {key} ({e})")

    break

April2024.pdf  :  b84b5eb4948f442c0ad455041509946e
[PDF] April2024.pdf | etag=b84b5eb4948f442c0ad455041509946e | pages_read=3/3
Page 1 of 3
duke-energy.com
800.521.2232
9101 5076 9698
Your Energy Bill
Service address  Bill date
Account number
May 6, 2024
30 days
For service   Apr 3 - May 2ALVAN DMELLO
418 S GRANT ST APT 12
BLOOMINGTON IN 47401
Billing summary
 Previous Amount Due $42.78
      Payment Received Apr 09 -42.78
 Current Electric Charges 38.80
 Taxes 2.72
 Total Amount Due May 28 $41.52
Your usage snapshot
Electric usage history
kWh 2023 2024
0
37
74
110
147
184
221
257
294
331
May Jun Jul Aug Sep Oct Nov Dec Jan Feb Mar Apr May
Average temperature in degrees
63o 70o 76o 73o 67o 56o 45o 40o 30o 41o 49o 58o 69o
Current Month May 2023 12-Month Usage Avg Monthly Usage
Electric (kWh) 193 0 N/A 237
12-month usage based on most recent history
Thank you for your payment.
 Current electric usage for meter number 318452714
Actual reading on May 2 
Previous reading on Apr 3
43438
- 43